![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2022-2023 </font>
</div>

    
<div align="center"> 

<font size=4>   </font>

<font size=5> Ejercicio 6 - Diseño de elementos Ópticos Difractivos </font>
</div>

- **Fecha**: 
        
- **Alumno**:  Alex Recuenco


# Introducción

En este ejercicio nos vamos a centrar en el diseño de elementos ópticos difractivos. Vamos a generar kinoformas, que luego pasaremos a máscaras binarias de amplitud o fase. En lugar de tener un algoritmo específico, geraremos máscaras de fase continúas y el paso final será, si es necesario, convertirlas en ampliftud y/o binarias.

- Diseño de máscaras.
- Verificaión del resultado.
- Grabación de máscaras como .bmp. Dichas máscaras nos servirán para la Práctica de laboratorio 3.

El algoritmo de propagación se proporcionará a través del archivo "IFTA.py".

Dentro de este archivo están las siguientes funciones:

- GS_far_algorithm. Permite realizar el algoritmo IFTA en campo lejano
- GS_near_algoritm. Permite realizar el algoritmo IFTA en campo cercano
- binarize. Realiza el proceso de binarización de la máscara generada
- nmse. Parámetro de calidad que compara la máscara objetivo y el resultado.

# Enunciados

1. **Diseño de una máscara en campo lejano.**

    a. Genere una distribución de intensidad objetivo compleja, target, a través de un .bmp o Scalar_mask_XY.
    
    b. Utilice el algoritmo GS_far_algorithm de IFTA.py para diseñar un DOE cuyo target sea la distribución de luz en campo lejano para la imagen propuesta. Dibuje la máscara generada. Dibuje la historia de error del algoritmo
    
    c. Determine la distribución de intensidad en campo lejano. Utilice la función verify_mask. 
  
    d. Determine el error comparando mediante la funcion mnse el objetivo (target) y el resultado. Grabe la máscara
          

2. **Diseño de una máscara en campo cercano**

    a. Sea una distribución objetivo formada por un único punto en el centro de la imagen (0,0). Dicha distribución de luz está ubicada a una distancia z = 15 mm de la máscara a calcular. Determine, con el algoritmo IFTA de campo cercano la máscara de fase. Calcule la intensidad obtenida por dicha máscara.
    
    b. Realice el mismo cálculo para una distribución objetivo de 2 puntos en el eje x equidistantes del centro (-a,0), (a,0).
    
3. **Comparación entre varios tipos de máscaras**

    a. Realice el mismo cálculo para una máscara cuadrada. 
    
    b. Compare el resultado para las configuraciones amplitud-continua, amplitud-binaria, fase-continua, fase-binaria

<div class="alert alert-block alert-success">
    
<b>Nota:</b>
    
En todos los casos, especialmente en las kinoformas de amplitud, es conveniente eliminar el orden 0 de la intensidad obtenida, pues reduce la visibilidad del resultado.

    - Para la función fft, hay un parámetro específico.

    - Al usar verify_mask en campo lejano, escriba z=0 o z=None
    
</div>



Proponga un valor para los parámetros que no vengan en el enunciado. En todos los casos, los resultados son gráficas de intensidad y fase luminosa.

<div class="alert alert-block alert-info">

<b>Consejos:</b>
    
    - Para una mejor observación y rapidez en los cálculos, utilice una máscara de 512 x 512 pixeles.
    
    - El tamaño de la máscara puede ser 1 mm.
    

</div>

## Importar módulos 


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib
import matplotlib.cm as cm

from diffractio import degrees, mm, plt, sp, um, np
from diffractio.scalar_fields_XY import Scalar_field_XY
from diffractio.utils_drawing import draw_several_fields
from diffractio.scalar_masks_XY import Scalar_mask_XY
from diffractio.scalar_sources_XY import Scalar_source_XY


In [ ]:
from IFTA import GS_far_algorithm, GS_Fresnel_algorithm, binarize, nmse, insert_pupil, verify_mask

In [ ]:
from dataclasses import dataclass


@dataclass
class MaskOptions:
    continuous: Scalar_mask_XY
    continuous_phase: Scalar_mask_XY
    binary: Scalar_mask_XY
    binary_phase: Scalar_mask_XY


def get_all_masks(continuous_mask):
    binary = binarize(continuous_mask)

    continuous_phase = np.exp(2 * np.pi * continuous_mask)

    # 0 -> 0, 1 -> pi
    binary_phase = np.exp(np.pi * continuous_mask)
    return MaskOptions(
        continuous=continuous_mask,
        binary=binary,
        continuous_phase=continuous_phase,
        binary_phase=binary_phase,
    )


# Ejercicio 1

**a) Genere una distribución de intensidad objetivo que desee, target, a través de un dibujo o Scalar_mask_XY.**

In [ ]:
size = 1 * mm
num_x = num_y = 512
wavelength = 0.5 * um

num_steps = 50


In [ ]:
x = np.linspace(-size // 2, size // 2, num_x)
y = np.linspace(-size // 2, size // 2, num_y)

source = Scalar_source_XY(x, y, wavelength)
source.plane_wave(A=1, theta=0, phi=0)

target = Scalar_mask_XY(x, y, wavelength)
target.image("masks/mascara1024.png")
target.draw()


**b) Utilice el algoritmo GS_far_algorithm** 

In [ ]:

mask_final, errors = GS_far_algorithm(None, target, num_steps)
print(errors)

**c) Verificación de resultados**

In [ ]:
# print(mask_final.u)
# def draw_mask_result(mask, **kwargs):
#     mask.draw(**kwargs)
#     plt.show()

#     source = Scalar_source_XY(x, y, wavelength=wavelength)
#     source.plane_wave()
#     input = source * mask

#     output = input.fft(new_field=True)
#     output.draw(**kwargs)
#     plt.show()

# print(mask_final.u)
# draw_mask_result(mask_final)

DOE:Scalar_mask_XY
DOE, result = verify_mask(mask_final, 0, False, False, True)

DOE.draw()
plt.show()
result.draw()
plt.show()



**d. Determine el error comparando mediante la funcion mnse el objetivo (target) y el resultado. Grabe la máscara**


In [ ]:
# print(target.u)
# difference = np.abs(output.u)**2-np.abs(target.u)**2
# print(difference)
# #rmse =float(np.mean(difference**2))**0.5
nmse_gs= nmse(np.abs(DOE.u)**2, np.abs(target.u)**2)
print(nmse_gs, errors[-1])

## QUESTION: Porque el error aqui es distinto entre ambos? No se supone que es el NMSE el que se esta usando 

**e. Grabar máscaras**

In [ ]:
DOE.save_mask('solutions/GS_far_algorithm.bmp')

# Ejercicio 2

In [ ]:
from dataclasses import dataclass, field
from typing import Tuple


@dataclass(frozen=True)
class DoeSettings:
    size: float = 1 * mm
    num_x: int = 512
    num_y: int = 512
    wavelength: float = 0.6328 * um

    z_distance: float = 15 * mm
    num_steps: float = 10


def obtain_dots_mask(target_factory, settings: DoeSettings = DoeSettings()):
    x = np.linspace(-settings.size / 2, settings.size / 2, settings.num_x)
    y = np.linspace(-settings.size / 2, settings.size / 2, settings.num_y)
    target = target_factory(x, y, settings.wavelength)

    mask_final, errors = GS_Fresnel_algorithm(
        source=None, target=target, z=settings.z_distance, num_steps=settings.num_steps
    )
    return settings, target, mask_final, errors


def plot_dots_target_mask(target, mask, errors):
    # target.circle(r0=(0,0),radius=5*um)
    target.draw()
    plt.title("Target image")
    plt.show()

    print(errors)

    ## QUESTION 2: Why are all the errors exactly the same

    mask.draw()
    plt.title("Mask")
    plt.show()


def plot_verification(mask: Scalar_mask_XY, *args, **kwargs):
    binary_mask = mask.binarize(bin_level=0.5, new_field=True)

    doe_matrix = [
        {
            "mask": mask,
            "has_mask": False,
            "is_phase": False,
            "is_binary": False,
            "title": "DOE continuous",
        },
        {
            "mask": mask,
            "has_mask": False,
            "is_phase": True,
            "is_binary": False,
            "title": "DOE continuous phase",
        },
        {
            "mask": binary_mask,
            "has_mask": False,
            "is_phase": False,
            "is_binary": True,
            "title": "DOE binary",
        },
        {
            "mask": binary_mask,
            "has_mask": False,
            "is_phase": True,
            "is_binary": True,
            "title": "DOE binary phase",
        },
    ]
    for opts in doe_matrix:
        title = opts.pop("title", "DOE")

        DOE, result = verify_mask(*args, **opts, **kwargs)
        DOE.draw()
        plt.title(title)
        plt.show()
        result.draw(logarithm=True)
        plt.title(f"Result of {title} (log scale)")
        plt.show()


## a) 1 punto

In [ ]:
def dot_target_factory(dots=(0, 0)):
    def factory(x, y, wavelength):
        target = Scalar_mask_XY(x, y, wavelength)
        # target.circle(r0=(0,0),radius=5*um)
        target.dots(r0=dots)
        return target
    return factory


In [ ]:
settings, target, mask_final, errors = obtain_dots_mask(dot_target_factory())

In [ ]:
plot_dots_target_mask(target, mask_final, errors)

**b) Verificación de la máscara**

In [ ]:
plot_verification(mask_final, z=settings.z_distance)

### Save masks

In [ ]:
mask_final.save_mask('solutions/GS_near_one_dot.bmp')

## b) 2 puntos

In [ ]:
dots=([-250, 250],[0,0])


settings, target, mask_final, errors = obtain_dots_mask(dot_target_factory(dots))

In [ ]:
plot_dots_target_mask(target, mask_final, errors)

**b) Verificación de la máscara**

In [ ]:
plot_verification(mask_final, z=settings.z_distance)

### Save mask

In [ ]:
mask_final.save_mask('solutions/GS_near_two_dots.bmp')

## Ejercicio 3.

**Realice el mismo cálculo para una máscara cuadrada.**


In [ ]:
def square_mask_factory(x,y,wavelength):
    target=Scalar_mask_XY(x,y,wavelength)
    target.square(r0=(0,0), size=100*um, angle=0)
    return target


In [ ]:
settings, target, mask_final, errors = obtain_dots_mask(square_mask_factory, DoeSettings(num_steps=100))

In [ ]:
plot_dots_target_mask(target, mask_final, errors)

In [ ]:

plot_verification(mask_final, z=settings.z_distance)


NOTE:

Notese como en los dibujos de fase, el dibujo parece que no tenga nada, esto es simplemente porque sólo tiene cambios de fase, y estamos representando la amplitud en este dibujo.

### Save mask

In [ ]:
mask_final.save_mask('solutions/GS_near_square.bmp');